In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoConfig,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)

In [2]:
RUS = "<RUS>"
UDM = "<UDM>"
KOMI = "<KOMI>"
MHR = "<MHR>"
EOS = "<eos>"
MANS = "<MANS>"
TRANSLATE = "<TRANSLATE>"
COMPUTE = "<COMPUTE>"
VOCAB = "<VOCAB>"

In [3]:
COMPUTE_COUNT = 50

In [21]:
dt = pd.read_csv("/home/maxim/Documents/projects/Task10/dataset/bigTrain/trainVOCAB.csv")

In [46]:
tokenizer.encode("<pad><pad><eos>")

[2, 0, 0, 1]

In [47]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_bos_token',
 '_add_eos_token',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_compile_jinja_template',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strateg

In [63]:
tokenizer._add_bos_token

False

In [62]:
tokenizer._add_tokens

<bound method PreTrainedTokenizerFast._add_tokens of GemmaTokenizerFast(name_or_path='google/gemma-2-2b', vocab_size=256000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<RUS>', '<UDM>', '<KOMI>', '<MHR>', '<MANS>', '<TRANSLATE>', '<COMPUTE>', '<VOCAB>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<bos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=F

In [64]:
tokenizer.add_special_tokens

False

In [65]:
tokenizer._add_eos_token, tokenizer.add_special_tokens

(False, False)

In [70]:
tokenizer("<MANS>")

{'input_ids': [270793], 'attention_mask': [1]}

In [4]:
def prepare_dataset(path):
    dt = pd.read_csv(path)
    source = dt.source.tolist()
    target = dt.target.tolist()
    len_ = len(source)
    data = []
    for i in range(len_ // 2):
        splited = source[i].split(VOCAB)
        if len(splited) == 2:
            vocab = splited[0]
            s_ = splited[1]
        else:
            vocab = ""
            s_ = splited[0]           
        data.append(VOCAB + vocab + VOCAB + MANS + s_ + TRANSLATE + COMPUTE_COUNT * COMPUTE + RUS + target[i] + EOS)

    for i in range(len_ // 2, len_):
        splited = source[i].split(VOCAB)
        if len(splited) == 2:
            vocab = splited[0]
            s_ = splited[1]
        else:
            vocab = ""
            s_ = splited[0]           
        data.append(VOCAB + vocab + VOCAB + RUS + s_ + TRANSLATE + COMPUTE_COUNT * COMPUTE + MANS + target[i] + EOS)
    return data

In [28]:
data = prepare_dataset("/home/maxim/Documents/projects/Task10/dataset/bigTrain/trainVOCAB.csv")
data.extend(prepare_dataset("/home/maxim/Documents/projects/Task10/dataset/bigTrain/valVOCAB.csv"))

data_test = prepare_dataset("/home/maxim/Documents/projects/Task10/dataset/bigTrain/testVOCAB.csv")

In [74]:
data_train = pd.concat([
    pd.read_csv("/home/maxim/Documents/projects/Task10/dataset/bigTrain/trainVOCAB.csv"), 
    pd.read_csv("/home/maxim/Documents/projects/Task10/dataset/bigTrain/valVOCAB.csv")])

In [82]:
data_test = pd.read_csv("/home/maxim/Documents/projects/Task10/dataset/bigTrain/testVOCAB.csv")

In [80]:
s = data_train.source.tolist()
t = data_train.target.tolist()

s = ["@@@".join([s[i], t[i]]) for i in range(len(s))]

In [85]:
s_ = data_test.source.tolist()
t_ = data_test.target.tolist()

t = ["@@@".join([s_[i], t_[i]]) for i in range(len(s_))]

In [86]:
len(s), len(t)

(155784, 6000)

In [78]:
data_train.source.tolist()

['о̄луӈкве - жить, быть, находиться, быть  иметься, жить  содержаться  находиться, состоять | пуӈк - голова, зуб, зубной<VOCAB>Пуӈке ке мēкыс о̄лыс:',
 'акваг - всегда, повсюду, постоянно, навсегда, везде, всюду | хо̄тал - день, солнце | минуӈкве - идти, ехать, лететь, пойти, уходить | ва̄ти - короткий, краткий | хоса - длинный, далёкий, долго, длинный  далёкий | э̄т - ночь<VOCAB>Хоса ман вāти минасыт, э̄тэ минэ̄гт, хо̄талэ минэ̄гт акваг.',
 '<VOCAB>Тувл хо̄талта̄лн.',
 'пы̄г - сын  парень  юноша | ёмас - добро, добрый, хороший, приятный, хорошо, приятно | кол - дом<VOCAB>А̄гиӈ-пыгыӈ ёмас кол.',
 'минуӈкве - идти, ехать, лететь, пойти, уходить<VOCAB>Минасум.',
 'ам - я, мой | ма̄ - земля, место, страна, земля  место  местность  страна | ма̄н - мы, нам, наш<VOCAB>Ма̄н юйпа̄лувт ул ня̄влхатэ̄н.',
 'пуссын - все, всё, весь | э̄лаль - вперёд, указывает на движение вперёд, дальше ()<VOCAB>Пуссын э̄лаль ё̄мттыянн!',
 'пун - перо (птичье), пух, мех, шерсть, пун | па̄л - половина  сторона, нар

In [29]:
data[24451] = "|".join(data[24451].split("|")[:2] + data[24451].split("|")[12:])
data[99343] = "|".join(data[99343].split("|")[:2] + data[99343].split("|")[36:])

In [17]:
len(data)

155784

In [73]:
len(data[0].split(COMPUTE))

51

In [15]:
data[100000]

'<VOCAB>бьется его сердце - сыме путги | жить - акват-о̄луӈкве, о̄луӈкве | жить счастливо - сюнюӈкве | жить отдельно - я̄нас о̄луӈкве | жить вместе - акват-о̄луӈкве | быть  иметься, жить  содержаться  находиться - о̄луӈкве | до настоящего времени - аньмус | до сего времени - аньтыгмус | довести до истощения (о животном) - повырмаптуӈкве | до тех пор - тувмус | до свидания - о̄с ёмас ӯлум, о̄съёмасӯлум | медлительный , медленный  тихий - ла̄сял | где - хōт | идти (не спеша) - ё̄мантаӈкве | идти пешком - ё̄муӈкве | идти - йиӈкве, йисинта̄луӈкве, йисинтаӈкве, минуӈкве, йиӈкве (ам ювум, нан ювын, нэн ювын, нан ювын) | идти по следу - койхатуӈкве | идти (о дожде, о дожде) - раквуӈкве | идти домой - юв-минуӈкве | идти рядом - аквма̄гылта̄рс минуӈкве | идти (о снеге) - тӯянтаӈкве | идти (домой) - юв-минуӈкве | нитки из жил - та̄н | убежать далеко - э̄л-ха̄йтуӈкве | далеко - э̄лн, хосан, хосат | указывает на движение вперёд, далеко в сторону () - э̄л-э̄ла | лесной - во̄р | охотиться (на диких 

In [3]:
dt

,Unnamed: 0,source,target
0,0,"о̄луӈкве - жить, быть, находиться, быть иметь...",Если голова его ослабнет:
1,1,"акваг - всегда, повсюду, постоянно, навсегда, ...","Долго ли, коротко ли едут, всю ночь едут, днём..."
2,2,<VOCAB>Тувл хо̄талта̄лн.,Оттуда сияй.
3,3,"пы̄г - сын парень юноша | ёмас - добро, добр...",С дочками-сыновьями хороший дом.
4,4,"минуӈкве - идти, ехать, лететь, пойти, уходить...",Поехала.
...,...,...,...
149779,149779,вблизи - ляпат<VOCAB>Я на Камчатке вблизи Бери...,– Ам Камчатка ма̄т Берингов пролив ляпат самын...
149780,149780,до настоящего времени - аньмус | до сего време...,Кинат суссылтан мус э̄тпос арыгтем хультыс.
149781,149781,"печь - кӯр | топить (печь) - па̄лтыглаӈкве, па...",Тӯяг ēмтум ма̄гыс кӯраныл па̄лтыглан ма̄ныл по...
149782,149782,"только что - аньты, усь, усьты | только - топ,...","Туп ка̄таге мось , э̄ртум то̄ргысыг."


In [18]:
len(items)

149784

In [69]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b", add_bos_token=False,)
with open("train/spm_man.vocab", "r+") as file1:
    vocab = [elem.split("\t")[0] for elem in file1.read().split("\n")]

RUS = "<RUS>"
UDM = "<UDM>"
KOMI = "<KOMI>"
MHR = "<MHR>"
MANS = "<MANS>"
TRANSLATE = "<TRANSLATE>"
COMPUTE = "<COMPUTE>"
VOCAB = "<VOCAB>"
special_tokens_dict = {
    "additional_special_tokens": [RUS, UDM, KOMI, MHR, MANS, TRANSLATE, COMPUTE, VOCAB]
}
# vocab.extend([RUS, UDM, KOMI, MHR, MANS])
new_tokens = set(vocab) - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens))
tokenizer.add_special_tokens(special_tokens_dict)

8

In [8]:
items = dt.source.tolist()

In [16]:
max_ = 0
for i in tqdm(range(len(items))):
    max_ = max(len(tokenizer.encode(items[i])), max_)

  0%|          | 0/149784 [00:00<?, ?it/s]

In [30]:
max_ = 0
for i in tqdm(range(len(data))):
    if len(tokenizer.encode(data[i])) > 2048:
        print(i)
    max_ = max(len(tokenizer.encode(data[i])), max_)

  0%|          | 0/155784 [00:00<?, ?it/s]

In [26]:
len(tokenizer.encode("|".join(data[99343].split("|")[:2] + data[99343].split("|")[36:])))

2042

In [31]:
# "|".join(data[99343].split("|")[:2] + data[99343].split("|")[36:])

In [32]:
# data[99343]

In [33]:
max_

2042

In [34]:
max_ = 0
for i in tqdm(range(len(data_test))):
    max_ = max(len(tokenizer.encode(data_test[i])), max_)

  0%|          | 0/6000 [00:00<?, ?it/s]

In [35]:
max_

1170

In [44]:
# tokenizer.encode(data[0])

In [36]:
import random

In [37]:
random.seed(777)
random.shuffle(data)
random.shuffle(data_test)

In [38]:
data.extend([data_test[i] for i in range(4000)])
data_test = data_test[4000:]

In [39]:
u = set.intersection(set(data), set(data_test))
data_test = list(set(data_test) - u)
random.shuffle(data_test)

In [65]:
assert len(data) == len(set(data) - set(data_test))

AssertionError: 

In [50]:
len(set(data) - set(data_test))

159058

In [42]:
dt = {}
dt['train'] = data
dt['test'] = data_test

In [44]:
import json
out_file = open("dataTranslate.json", "w")  
    
json.dump(dt, out_file, indent = 6)  
    
out_file.close()

In [40]:
len(data_test), len(data), len(data) - len(data_test)

(1986, 159784, 157798)

In [ ]:
set(data) set(data_test)

In [41]:
u = set.intersection(set(data), set(data_test))
len(u)

0

In [66]:
len(set(data).difference(set(data_test)))

159058

In [42]:
list(u)[0] in data

True

In [43]:
list(u)[0] in data_test

True

In [70]:
for i in tqdm(range(len(data_test))):
    if data_test[i] in data:
        print(i)

  0%|          | 0/1986 [00:00<?, ?it/s]

In [89]:
random.seed(777)
random.shuffle(s)
random.shuffle(t)

In [90]:
s.extend([t[i] for i in range(4000)])
t = t[4000:]

In [91]:
u = set.intersection(set(s), set(t))
print(len(u))
t = list(set(t) - u)
random.shuffle(t)

14


In [93]:
len(t), len(s)

(1986, 159784)

In [94]:
source = [elem.split("@@@")[0] for elem in s]
target = [elem.split("@@@")[1] for elem in s]

In [104]:
b = 3
source[b], target[b]

('ат - не, пять, запах, вкус | нэ̄мат - никакой | хо̄тпа - кто-то, некто<VOCAB>Нэ̄мат хо̄тпа ат лю̄ньщыс.',
 'Что-то никто не плакал.')

In [100]:
pd.DataFrame({"source": source, "target": target}).to_csv("trainBig.csv")

In [101]:
source = [elem.split("@@@")[0] for elem in t]
target = [elem.split("@@@")[1] for elem in t]

In [103]:
pd.DataFrame({"source": source, "target": target}).to_csv("testBig.csv")